# 家庭關係本體

此範例是 [AI for Beginners Curriculum](http://github.com/microsoft/ai-for-beginners) 的一部分，靈感來自於[這篇部落格文章](https://habr.com/post/270857/)。

我總是覺得記住家庭中人與人之間的不同關係很困難。在這個範例中，我們將使用一個定義家庭關係的本體，以及實際的家譜樹，來展示如何進行自動推理以找出所有親屬。

### 獲取家譜樹

作為範例，我們將使用[羅曼諾夫沙皇家族](https://en.wikipedia.org/wiki/House_of_Romanov)的家譜樹。描述家庭關係最常見的格式是 [GEDCOM](https://en.wikipedia.org/wiki/GEDCOM)。我們將使用 GEDCOM 格式的羅曼諾夫家族樹：


In [1]:
!head -15 data/tsars.ged

0 HEAD
1 CHAR UTF8
1 GEDC
2 VERS 5.5
0 @0@ INDI
1 NAME Mihail Fedorovich /Romanov/
1 SEX M
1 BIRT
2 DATE 1613
1 DEAT 
2 DATE 1645
1 FAMS @41@
0 @1@ INDI
1 NAME Evdokija Lukjanovna /Streshneva/
1 SEX F


要使用 GEDCOM 文件，我們可以使用 `python-gedcom` 庫：


In [2]:
import sys
!{sys.executable} -m pip install python-gedcom

此程式庫解決了一些與檔案解析相關的技術問題，但仍然提供了對樹中所有個人和家庭的相當低層次的訪問。以下是我們如何解析檔案並顯示所有個人列表的方法：


In [3]:
from gedcom.parser import Parser
from gedcom.element.individual import IndividualElement
from gedcom.element.family import FamilyElement
g = Parser()
g.parse_file('data/tsars.ged')

In [4]:
d = g.get_element_dictionary()
[ (k,v.get_name()) for k,v in d.items() if isinstance(v,IndividualElement)]

[('@0@', ('Mihail Fedorovich', 'Romanov')),
 ('@1@', ('Evdokija Lukjanovna', 'Streshneva')),
 ('@2@', ('Aleksej Mihajlovich', 'Romanov')),
 ('@3@', ('Marija Ilinichna', 'Miloslavskaja')),
 ('@4@', ('Natalja Kirillovna', 'Naryshkina')),
 ('@5@', ('Marfa Matveevna', 'Apraksina')),
 ('@6@', ('Fedor Alekseevich', 'Romanov')),
 ('@7@', ('Sofja Aleksevna', 'Romanova')),
 ('@8@', ('Ivan V Alekseevich', 'Romanov')),
 ('@9@', ('Praskovja Fedorovna', 'Saltykova')),
 ('@10@', ('Ekaterina Ivanovna', 'Romanova')),
 ('@11@', ('Anna Ivanovna', 'Romanova')),
 ('@12@', ('Fridrih Vilgelm', 'Kurlandskij')),
 ('@13@', ('Karl Leopold', 'Meklenburg-Shverinskij')),
 ('@14@', ('Anna Leopoldovna', 'Meklenburg-Shverinskaja')),
 ('@15@', ('Anton Ulrih', 'Braunshvejg-Volfenbjuttelskij')),
 ('@16@', ('Ivan VI Antonovich', 'Braunshvejg-Volfenbjuttelskij')),
 ('@17@', ('Petr I Alekseevich', 'Romanov')),
 ('@18@', ('Evdokija Fedorovna', 'Lopuhina')),
 ('@19@', ('Ekaterina I Alekseevna', 'Mihajlova')),
 ('@20@', ('Ale

以下是我們如何獲取有關家庭的信息。請注意，這會給我們一個**標識符**的列表，如果我們想要更清楚的資訊，我們需要將它們轉換為名稱：


In [5]:
d = g.get_element_dictionary()
[ (k,[x.get_value() for x in v.get_child_elements()]) for k,v in d.items() if isinstance(v,FamilyElement)]

[('@41@', ['@0@', '@1@', '@2@']),
 ('@42@', ['@2@', '@3@', '@6@', '@7@', '@8@']),
 ('@43@', ['@8@', '@9@', '@10@', '@11@']),
 ('@44@', ['@13@', '@10@', '@14@']),
 ('@45@', ['@15@', '@14@', '@16@']),
 ('@46@', ['@2@', '@4@', '@17@']),
 ('@47@', ['@17@', '@18@', '@20@']),
 ('@48@', ['@20@', '@21@', '@22@']),
 ('@49@', ['@17@', '@19@', '@23@', '@24@']),
 ('@50@', ['@25@', '@23@', '@26@']),
 ('@51@', ['@26@', '@27@', '@28@']),
 ('@52@', ['@28@', '@30@', '@31@', '@33@']),
 ('@53@', ['@33@', '@34@', '@35@']),
 ('@54@', ['@35@', '@36@', '@37@']),
 ('@55@', ['@37@', '@38@', '@39@'])]

### 獲取家庭本體

接下來，我們來看看[家庭本體](https://raw.githubusercontent.com/blokhin/genealogical-trees/master/data/header.ttl)，它被定義為一組語義網三元組。這個本體定義了像 `isUncleOf`、`isCousinOf` 等許多關係。所有這些關係都是基於基本謂詞 `isMotherOf`、`isFatherOf`、`isBrotherOf` 和 `isSisterOf` 定義的。我們將使用自動推理來利用該本體推導出所有其他關係。

以下是 `isAuntOf` 屬性的範例定義，它被定義為 `isSisterOf` 和 `isParentOf` 的組合（*姑姑或阿姨是某人父母的姐妹*）。

```
fhkb:isAuntOf a owl:ObjectProperty ;
    rdfs:domain fhkb:Woman ;
    rdfs:range fhkb:Person ;
    owl:propertyChainAxiom ( fhkb:isSisterOf fhkb:isParentOf ) .
```


In [6]:
!head -20 data/onto.ttl

@prefix fhkb: <http://www.example.com/genealogy.owl#> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix xml: <http://www.w3.org/XML/1998/namespace> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

<http://www.example.com/genealogy.owl#> a owl:Ontology .

fhkb:DomainEntity a owl:Class .

fhkb:Man a owl:Class ;
    owl:equivalentClass [ a owl:Class ;
            owl:intersectionOf ( fhkb:Person [ a owl:Restriction ;
                        owl:onProperty fhkb:hasSex ;
                        owl:someValuesFrom fhkb:Male ] ) ] .

fhkb:Woman a owl:Class ;
    owl:equivalentClass [ a owl:Class ;
            owl:intersectionOf ( fhkb:Person [ a owl:Restriction ;


### 建構推理本體

為了簡化，我們將建立一個本體文件，其中包含家庭本體的原始規則，以及來自我們 GEDCOM 文件的個人事實。我們將逐步處理 GEDCOM 文件，提取有關家庭和個人的信息，並將其轉換為三元組。


In [7]:
!cp data/onto.ttl .

gedcom_dict = g.get_element_dictionary()
individuals, marriages = {}, {}

def term2id(el):
    return "i" + el.get_pointer().replace('@', '').lower()

out = open("onto.ttl","a")

for k, v in gedcom_dict.items():
    if isinstance(v,IndividualElement):
        children, siblings = set(), set()
        idx = term2id(v)

        title = v.get_name()[0] + " " + v.get_name()[1]
        title = title.replace('"', '').replace('[', '').replace(']', '').replace('(', '').replace(')', '').strip()

        own_families = g.get_families(v, 'FAMS')
        for fam in own_families:
            children |= set(term2id(i) for i in g.get_family_members(fam, "CHIL"))

        parent_families = g.get_families(v, 'FAMC')
        if len(parent_families):
            for member in g.get_family_members(parent_families[0], "CHIL"): # NB adoptive families i.e len(parent_families)>1 are not considered (TODO?)
                if member.get_pointer() == v.get_pointer():
                    continue
                siblings.add(term2id(member))

        if idx in individuals:
            children |= individuals[idx].get('children', set())
            siblings |= individuals[idx].get('siblings', set())
        individuals[idx] = {'sex': v.get_gender().lower(), 'children': children, 'siblings': siblings, 'title': title}

    elif isinstance(v,FamilyElement):
        wife, husb, children = None, None, set()
        children = set(term2id(i) for i in g.get_family_members(v, "CHIL"))

        try:
            wife = g.get_family_members(v, "WIFE")[0]
            wife = term2id(wife)
            if wife in individuals: individuals[wife]['children'] |= children
            else: individuals[wife] = {'children': children}
        except IndexError: pass
        try:
            husb = g.get_family_members(v, "HUSB")[0]
            husb = term2id(husb)
            if husb in individuals: individuals[husb]['children'] |= children
            else: individuals[husb] = {'children': children}
        except IndexError: pass

        if wife and husb: marriages[wife + husb] = (term2id(v), wife, husb)

for idx, val in individuals.items():
    added_terms = ''
    if val['sex'] == 'f':
        parent_predicate, sibl_predicate = "isMotherOf", "isSisterOf"
    else:
        parent_predicate, sibl_predicate = "isFatherOf", "isBrotherOf"
    if len(val['children']):
        added_terms += " ;\n    fhkb:" + parent_predicate + " " + ", ".join(["fhkb:" + i for i in val['children']])
    if len(val['siblings']):
        added_terms += " ;\n    fhkb:" + sibl_predicate + " " + ", ".join(["fhkb:" + i for i in val['siblings']])
    out.write("fhkb:%s a owl:NamedIndividual, owl:Thing%s ;\n    rdfs:label \"%s\" .\n" % (idx, added_terms, val['title']))

for k, v in marriages.items():
    out.write("fhkb:%s a owl:NamedIndividual, owl:Thing ;\n    fhkb:hasFemalePartner fhkb:%s ;\n    fhkb:hasMalePartner fhkb:%s .\n" % v)

out.write("[] a owl:AllDifferent ;\n    owl:distinctMembers (")
for idx in individuals.keys():
    out.write("    fhkb:" + idx)
for k, v in marriages.items():
    out.write("    fhkb:" + v[0])
out.write("    ) .")
out.close()

In [8]:
!tail onto.ttl

    fhkb:hasFemalePartner fhkb:i34 ;
    fhkb:hasMalePartner fhkb:i33 .
fhkb:i54 a owl:NamedIndividual, owl:Thing ;
    fhkb:hasFemalePartner fhkb:i36 ;
    fhkb:hasMalePartner fhkb:i35 .
fhkb:i55 a owl:NamedIndividual, owl:Thing ;
    fhkb:hasFemalePartner fhkb:i38 ;
    fhkb:hasMalePartner fhkb:i37 .
[] a owl:AllDifferent ;
    owl:distinctMembers (    fhkb:i0    fhkb:i1    fhkb:i2    fhkb:i3    fhkb:i4    fhkb:i5    fhkb:i6    fhkb:i7    fhkb:i8    fhkb:i9    fhkb:i10    fhkb:i11    fhkb:i12    fhkb:i13    fhkb:i14    fhkb:i15    fhkb:i16    fhkb:i17    fhkb:i18    fhkb:i19    fhkb:i20    fhkb:i21    fhkb:i22    fhkb:i23    fhkb:i24    fhkb:i25    fhkb:i26    fhkb:i27    fhkb:i28    fhkb:i29    fhkb:i30    fhkb:i31    fhkb:i32    fhkb:i33    fhkb:i34    fhkb:i35    fhkb:i36    fhkb:i37    fhkb:i38    fhkb:i39    fhkb:i40    fhkb:i41    fhkb:i42    fhkb:i43    fhkb:i44    fhkb:i45    fhkb:i46    fhkb:i47    fhkb:i48    fhkb:i49    fhkb:i50    fhkb:i51    fhkb:i52    fhkb:i53    fhkb:

### 執行推理

現在我們希望能夠使用這個本體進行推理和查詢。我們將使用 [RDFLib](https://github.com/RDFLib)，這是一個用於以不同格式讀取 RDF 圖、查詢等的庫。

對於邏輯推理，我們將使用 [OWL-RL](https://github.com/RDFLib/OWL-RL) 庫，該庫允許我們構建 RDF 圖的**閉包**，也就是說，添加所有可以推導出的概念和關係。


In [10]:
!{sys.executable} -m pip install rdflib
!{sys.executable} -m pip install git+https://github.com/RDFLib/OWL-RL.git

  Cloning https://github.com/RDFLib/OWL-RL.git to /tmp/pip-req-build-lbfzwi3m
  Running command git clone --filter=blob:none --quiet https://github.com/RDFLib/OWL-RL.git /tmp/pip-req-build-lbfzwi3m
  Resolved https://github.com/RDFLib/OWL-RL.git to commit a77e1791b88b54aace609bc6000aac14c7add4ff
  Preparing metadata (setup.py) ... done


讓我們打開本體文件，看看它包含了多少三元組：


In [11]:
import rdflib
from owlrl import DeductiveClosure, OWLRL_Extension

g = rdflib.Graph()
g.parse("onto.ttl", format="turtle")

print("Triplets found:%d" % len(g))

Triplets found:669


現在讓我們構建閉包，看看三元組的數量如何增加：


In [12]:
DeductiveClosure(OWLRL_Extension).expand(g)
print("Triplets after inference:%d" % len(g))

Triplets after inference:4246


### 查詢親屬關係

現在我們可以查詢圖譜來查看人與人之間的不同關係。我們可以結合使用 **SPARQL** 語言和 `query` 方法。在我們的例子中，讓我們來看看家譜中所有的 **叔叔**：


In [13]:
qres = g.query(
    """SELECT DISTINCT ?aname ?bname
       WHERE {
          ?a fhkb:isUncleOf ?b .
          ?a rdfs:label ?aname .
          ?b rdfs:label ?bname .
       }""")

for row in qres:
    print("%s is uncle of %s" % row)

Fedor Alekseevich Romanov is uncle of Ekaterina Ivanovna Romanova
Aleksandr I Pavlovich Romanov is uncle of Aleksandr II Nikolaevich Romanov
Fedor Alekseevich Romanov is uncle of Anna Ivanovna Romanova


隨意嘗試不同的家庭關係。例如，您可以查看 `isAncestorOf` 關係，它會遞迴地定義某個人的所有祖先。

最後，讓我們整理一下！


In [14]:
!rm onto.ttl


---

**免責聲明**：  
本文件使用 AI 翻譯服務 [Co-op Translator](https://github.com/Azure/co-op-translator) 進行翻譯。儘管我們努力確保翻譯的準確性，但請注意，自動翻譯可能包含錯誤或不準確之處。原始文件的母語版本應被視為權威來源。對於關鍵資訊，建議使用專業人工翻譯。我們對因使用此翻譯而引起的任何誤解或錯誤解釋不承擔責任。
